# Парсер курсов валют с сайта ЦБ

*Данный парсер собирает данные с сайта ЦБ о курсах валют с июля 1998 года по месяцам и выводит данные в виде таблицы в Excel файл*
- Для примера взят период за год
- В парсере есть временные ограничения, чтобы сайт не заблокировал подозрительную активность

In [1]:
import time
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from xml.dom import minidom
import pandas as pd
import datetime as DT

In [2]:
URL = 'https://www.cbr.ru/currency_base/daily/?UniDbQuery.Posted=True&UniDbQuery.To=01.07.1992'

In [3]:
req = requests.get(URL)

In [4]:
soup = BeautifulSoup(req.text,'lxml')

In [5]:
table = soup.find('table')

In [6]:
heading_table = [] # Список, в котором будут храниться названия столбцов
for row in table.find_all('th'): # Названия столбцов прячутся в элементах th, 
# поэтому будем искать все элементы th внутри table и пробегать по ним в цикле
        heading_table.append(row.text) # Добавляем контент из тега th в список heading_table методом append()
print(heading_table)

['Цифр. код', 'Букв. код', 'Единиц', 'Валюта', 'Курс']


In [7]:
heading_table.append('Дата')

In [8]:
heading_table

['Цифр. код', 'Букв. код', 'Единиц', 'Валюта', 'Курс', 'Дата']

In [9]:
#content=[] # Список, в котором будут храниться данные из таблицы
#for row in table.find_all('tr'): 
# Каждая строка обрамляется тегом tr, необходимо пробежаться в цикле по всем строкам
    #if not row.find_all('th'): 
# Эта проверка необходима, чтобы пропустить первую строку таблицы с заголовками
           # content.append([element.text for element in row.find_all('td')])
            # В каждой строке контент ячейки обрамляется тегами <td> </td>
            # Необходимо пробежаться в цикле по всем элементам td, вычленить контент из ячеек и добавить его в список 
            # Затем добавить каждый из списков в список content 
#print(content) 

In [14]:
# generation dates
start_date = DT.datetime(1992,7,1)
end_date = DT.datetime(1993,7,31)

In [15]:
list_date = pd.date_range(min(start_date,end_date),max(start_date,end_date)).strftime('%d.%m.%Y')

In [16]:
content_2 = []
for row in range(0,len(list_date)):
    URL_1 = 'https://www.cbr.ru/currency_base/daily/?UniDbQuery.Posted=True&UniDbQuery.To={name}'.format(name=list_date[row])
    req_1 = requests.get(URL_1)
    soup_1 = BeautifulSoup(req_1.text,'lxml')
    table_1 = soup_1.find('table')
    content_1=[]
    for i in table_1.find_all('tr'): 
        if not i.find_all('th'): 
            content_1.append([element.text for element in i.find_all('td')])
    # Добавляем дату в список
    for j in range(0,len(content_1)): 
        content_1[j].insert(5,list_date[row])
    content_2 = content_2 + content_1
    time.sleep(30)

In [17]:
course_data_1 = pd.DataFrame(content_2, columns=heading_table) 
# в качестве данных передаем двумерный список content, а в качестве заголовков - heading_table
course_data 

,Цифр. код,Букв. код,Единиц,Валюта,Курс,Дата
0,036,AUD,1,Австралийский доллар,"93,7400",01.07.1992
1,040,ATS,1,Австрийский шиллинг,"11,6700",01.07.1992
2,056,BEF,10,Бельгийских франков,"39,9200",01.07.1992
3,300,GRD,1000,Греческих драхм,"6,7700",01.07.1992
4,208,DKK,1,Датская крона,"21,3900",01.07.1992
...,...,...,...,...,...,...
1409,826,GBP,1,Фунт стерлингов Соединенного королевства,"405,8400",31.08.1992
1410,752,SEK,1,Шведская крона,"39,7400",31.08.1992
1411,756,CHF,1,Швейцарский франк,"162,1200",31.08.1992
1412,954,XEU,1,ЭКЮ,"293,5000",31.08.1992


In [18]:
course_data_1.to_excel('./course_data.xlsx')